This notebook was created by Donna Faith Go.

## Imports

In [1]:
from typing import List, Optional
import math
from collections import deque

### Stone Game II
**Description**

Alice and Bob continue their games with piles of stones. There are a number of piles arranged in a row, and each pile has a positive integer number of stones piles[i]. The objective of the game is to end with the most stones.

Alice and Bob take turns, with Alice starting first.

On each player's turn, that player can take all the stones in the first X remaining piles, where 1 <= X <= 2M. Then, we set M = max(M, X). Initially, M = 1.

The game continues until all the stones have been taken.

Assuming Alice and Bob play optimally, return the maximum number of stones Alice can get.

**Example 1**

Input: piles = [2,7,9,4,4]

Output: 10

**Example 2**

Input: piles = [1,2,3,4,5,100]

Output: 104

I think that this problem can be solved with a dynamic programming problem where at each step, look at the maximum stones the current player can collect from `index` with current `M`.

This problem also seems to be a zero-sum game where Alice's gain is Bob's loss and vice versa. 
Hence, a valid solution would be trying to minimize Bob's gain to maximize Alice's win. 

Here, I tried to model the dp state as: dp[i, M] = max number of stones from piles[:i] with current M. 
There are two variables in the dp state as the condition we are trying to look for depends on two variables. 

In [7]:
class Solution:
    def stoneGameII(self, piles: List[int]) -> int:
        dp = {}
        
        def dfs(alice, i, M):
            if i == len(piles):
                return 0
            if (alice, i, M) in dp:
                return dp[(alice, i, M)]

            res = 0 if alice else float('inf')
            total = 0 

            for X in range(1, M * 2 + 1):
                if i + X > len(piles):
                    break
                
                total += piles[i + X - 1]
                
                if alice:
                    res = max(res, total + dfs(not alice, i + X, max(M, X)))
                else:
                    res = min(res, dfs(not alice, i + X, max(M, X)))

            dp[(alice, i, M)] = res
            return res
            
        return dfs(True, 0, 1)

s = Solution()
piles = [2, 7, 9, 4, 4]
s.stoneGameII(piles)

10

I tried to understand the problem better with the code below.
To be transparent, this code was generated by ChatGPT.

In [3]:
def explore(piles, i, M, player, indent=""):
    n = len(piles)
    if i >= n:
        print(indent + f"{player} has no piles left → gain 0")
        return 0

    # If can take all
    if i + 2*M >= n:
        total = sum(piles[i:])
        print(indent + f"{player} takes ALL remaining {piles[i:]} → gain {total}")
        return total

    total_suffix = sum(piles[i:])
    print(indent + f"{player}'s turn at (i={i}, M={M}), remaining piles = {piles[i:]}, total = {total_suffix}")

    best_gain = 0
    best_choice = None

    # Try all possible X
    for X in range(1, 2*M + 1):
        if i + X > n: 
            break
        taken = sum(piles[i:i+X])
        nextM = max(M, X)

        print(indent + f"  {player} tries taking X={X} → {piles[i:i+X]} (gain {taken})")

        # Opponent plays optimally
        opponent = "Alice" if player == "Bob" else "Bob"
        opponent_gain = explore(piles, i+X, nextM, opponent, indent + "    ")
        player_gain = total_suffix - opponent_gain

        print(indent + f"  If {player} takes X={X}, then {opponent} gets {opponent_gain}, so {player} ends with {player_gain}\n")

        if player_gain > best_gain:
            best_gain = player_gain
            best_choice = X

    print(indent + f"Best for {player} at (i={i}, M={M}) is taking X={best_choice} → gain {best_gain}\n")
    return best_gain


# Example drill
piles = [2, 7, 9, 4]
print("=== Start of Game ===")
alice_best = explore(piles, 0, 1, "Alice")
print(f"\nFinal Result: Alice can guarantee {alice_best} stones.")


=== Start of Game ===
Alice's turn at (i=0, M=1), remaining piles = [2, 7, 9, 4], total = 22
  Alice tries taking X=1 → [2] (gain 2)
    Bob's turn at (i=1, M=1), remaining piles = [7, 9, 4], total = 20
      Bob tries taking X=1 → [7] (gain 7)
        Alice takes ALL remaining [9, 4] → gain 13
      If Bob takes X=1, then Alice gets 13, so Bob ends with 7

      Bob tries taking X=2 → [7, 9] (gain 16)
        Alice takes ALL remaining [4] → gain 4
      If Bob takes X=2, then Alice gets 4, so Bob ends with 16

    Best for Bob at (i=1, M=1) is taking X=2 → gain 16

  If Alice takes X=1, then Bob gets 16, so Alice ends with 6

  Alice tries taking X=2 → [2, 7] (gain 9)
    Bob takes ALL remaining [9, 4] → gain 13
  If Alice takes X=2, then Bob gets 13, so Alice ends with 9

Best for Alice at (i=0, M=1) is taking X=2 → gain 9


Final Result: Alice can guarantee 9 stones.
